<a href="https://colab.research.google.com/github/ignazioa/python-api-example/blob/master/notebooks/Gait_patterns_TF_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Gait patterns classification with TF.Keras and Moveshelf API
         
Step-by-step walkthrough to build a basic (MLP) gait classifier that can be used to identify Diplegic Gait patterns, using Moveshelf, TensorFlow and Keras   
Visit https://moveshelf.com/movement-science to know more.    

*Copyright Moveshelf Labs B.V. 2019.*

####Goals: 
*   Build a gait pattern classifier using an existing Gait Analsysis dataset
    
####Prerequisites: 
*   Basic Python understanding ;-) 
*   A Moveshelf account with API key <br>(free signup on https://moveshelf.com/movement-science and instructions of https://github.com/moveshelf/python-api-example )

##Setup your environment

###Import dependencies

Simply copy Moveshelf Python API's example repository in a dedicated local directory.    
These API and helpers are used to securely access Movement Analysis data remotely stored on Moveshelf.

In [0]:
!git clone https://github.com/moveshelf/python-api-example.git mvshlfpy
!pip install mypy_extensions

Import required packages in this environment: basic numerical packages as well as [TensorFlow](https://www.tensorflow.org) and [Keras](https://keras.io/)

In [0]:
#Math and plot utils
import numpy as np
import matplotlib.pyplot as plt
import IPython

# TensorFlow and Keras
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras

from mvshlfpy import api, helpers, datasets

###Setup your Moveshelf API
Make sure your API key is uploaded within this environment **as file**  
(once downloaded from Moveshelf it is by default named 'mvshlf-api-key.json' and it can be uploaded under the 'Files' tab here on the left side, as showed below).![alt text](http://static.moveshelf.com/ms-cc/api_upload.png)    
Further instructions are available on https://forum.moveshelf.com/t/getting-an-api-key/32 

In [0]:
mvshlfapi = api.MoveshelfApi('mvshlf-api-key.json') #read more on https://forum.moveshelf.com/t/getting-an-api-key/32

##Prepare your data

###Load dataset
As anticipated we will use a prepopulated Diplegic gait patterns dataset available at: https://moveshelf.com/project/UHJvamVjdKZkEQa_4U15n6DPj_SgaMY  

In [0]:
projectId = "UHJvamVjdKZkEQa_4U15n6DPj_SgaMY" #it's simply the last part of the url https://moveshelf.com/project/<projectId>

cp01 = datasets.Dataset(mvshlfapi, projectId)
cp01.load_data()

###Define training and test sets

For fair evaluation of our model during tests, let's split the dataset in training and test sets (~80% training and 20% test)

In [0]:
(train_patterns, train_classes) = cp01.get_training_set()
(test_patterns, test_classes) = cp01.get_test_set()
classes_labels = cp01.get_labels()

print("Training set: " + str(train_patterns.shape))
print("Test set: " + str(test_patterns.shape))
print("Classes: " + str(classes_labels))

###Gait patterns definition
To train our classifier we will use only used normalized joint kinematics information. All axsis of all available joints are represented with respect to gait cycle percentage, and stacked together in a nice gait-pattern ;-) 

In [0]:
ax = plt.imshow(train_patterns[0, :, :])
ax = plt.title(classes_labels[int(train_classes[0])])
ax = plt.xlabel("Gait cycle %")
ax = plt.ylabel("Joint axis")

##Prepare your classifier

###Setup the model of our simple classifier

For this example we will build a basic MLP classifier using a standard [sequential model](https://keras.io/getting-started/sequential-model-guide/) that maps the joint kinematics values into a dense layer and finally to a output one.  

In [0]:
#basic classifier 
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(36, 100)), #maps input pattern
    keras.layers.Dense(25, activation=tf.nn.relu), #magic happens here
    keras.layers.Dropout(0.1), #helps avoiding overfitting on the (small) training set
    keras.layers.Dense(4, activation=tf.nn.softmax) #read more on https://keras.io/optimizers/ 
])

model.compile(optimizer=keras.optimizers.Adam(lr=0.00005),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

###Train the classifier

After all this preparation let's feed our training set to the optimizer to fit our model, in batches. 

In [0]:
history = model.fit(train_patterns, train_classes, epochs=200, batch_size=25)

###Test accuracy of the model with our test set

So far so good, training accuracy and loss functions suggests that the model is well trained but ... how do we know there is no overfit? Let's evaluate the model using our test set.  

In [0]:
test_loss, test_acc = model.evaluate(test_patterns, test_classes)

##Make your first prediction!

OK cool, the evaluation on the test set confirms that our model is doing rather well (right??) ...    
So, let's use one of the test patterns to see how our model's prediction look like.    
First select a gait session id from our test set and visually inspect the 3D marker data.

In [0]:
example = 0
clipId = cp01.get_test_ids()[example]

htmlstr ='<iframe src="https://moveshelf.com/embed/'+ clipId + '?autoplay=true" width="600px" height="350px" scrolling="no" frameborder="0" allowfullscreen allow="vr" style="margin-right: 10px"></iframe>'
IPython.display.HTML(htmlstr) #read more on https://forum.moveshelf.com/t/how-to-embed-moveshelf-on-your-webpage-and-on-this-forum/31

Did you recognize the pattern? Let's see if our model can do a similarly good job!

In [0]:
predictions = model.predict(test_patterns[[example], :, :])
n_classes = len(classes_labels)
plt.bar(range(n_classes), predictions[0], color='gray')
plt.xticks(range(n_classes), classes_labels, rotation=70)
if( np.argmax(predictions[0]) == test_classes[example]):
  color = '#25aee6' #prediction is correct
else:
  color = 'red' #prediction is wrong :(
ax = plt.bar(np.argmax(predictions[0]), np.max(predictions[0]), color=color)

##Conclusion
Well done! Your classifier is well trained and ready to be deployed in the wild and on Moveshelf ;-) 